In [33]:
import db as db
import sqlalchemy as sql
from db import *
%run util.py

hostname = "cc1"
db.init(hostname)

'mysql://cc1:3306/project_b'

# coCLgenResults

In [ ]:
TABLE_NAME = "CLgen"
TABLE = coCLgenResult

In [21]:
import cldrive
import sys

def get_co_report(result_id: int, table=coCLgenResult, report_type: str="bf"):
    with Session(commit=False) as s:
        result = s.query(table).filter(table.id == result_id).first()

        if not result:
            raise KeyError(f"no result with ID {result_id}")

        flags = result.params.to_flags()
        program = result.program

        # generate bug report
        now = datetime.datetime.utcnow().isoformat()

        bug_type = {
            "bf": "compilation failure",
            "bc": "compiler crash",
            "c": "runtime crash",
        }[report_type]

        report_id = crypto.md5_str(table.__name__) + "-" + str(result.id)
        result_output = '\n'.join([f'//   {line}' for line in result.stderr.strip().split('\n')
                                   if not line.startswith("+ ")])
        header = f"""
// bug report {report_id}.c
// 
// Metadata:
//   OpenCL platform:        {result.testbed.platform}
//   OpenCL device:          {result.testbed.device}
//   Driver version:         {result.testbed.driver}
//   OpenCL version:         {result.testbed.opencl}
//   Host operating system:  {result.testbed.host}
//   OpenCL optimizations:   {result.params.optimizations_on_off}
//
// Output:
{result_output}
//   [Return code {result.status}]
//\
"""
        if isinstance(program, CLgenProgram):
            src = s.query(CLgenHarness).filter(
                CLgenHarness.program_id == program.id,
                CLgenHarness.params_id == result.params.id).first().src
        else:
            src = cldrive.emit_c(
                env=cldrive.make_env(),
                src=result.program.src,
                compile_only=True,
                create_kernel=result.params.build_kernel,
                gsize=None, lsize=None, inputs=None,
            )
        return (header + src).strip()

Segfaults:

In [5]:
from collections import defaultdict
from labm8 import fs

outbox = fs.path("outbox", "coCLgenResults", "segfaults")
fs.mkdir(outbox)
with Session(commit=True) as s:
    q = s.query(TABLE)\
        .filter(TABLE.status == 139,
                sql.or_(TABLE.submitted == None, TABLE.submitted == 0),
                sql.or_(TABLE.dupe == None, TABLE.dupe == 0))\
        .order_by(TABLE.id)

    dupes = 0
    errs = set()
    for result in q:
        key = result.testbed_id, result.program_id

        if key in errs:
            dupes += 1
            result.dupe = 1
            continue
        errs.add(key)

        vendor = PLATFORMS_2_VENDORS[result.testbed.platform]
        outpath = fs.path(outbox, f"bug-report-{vendor}-{result.id}.c")            

        report = get_co_report(**{
            "result_id": result.id,
            "table": coCLgenResult,
        })

        with open(outpath, "w") as outfile:
            print(report, file=outfile)
        result.submitted = 1
        print(outpath)
print("done")
print(f"{dupes} duplicates flagged")

outbox/coCLgenResults/segfaults/bug-report-intel-69539.c
done
0 duplicates flagged


# CLGenResults

In [26]:
TABLE_NAME = "CLgen"
TABLE = CLgenResult

In [34]:
outbox = fs.path("outbox", "CLgenResults", "bc")
fs.mkdir(outbox)
with Session(commit=False) as s:
    q = s.query(TABLE)\
        .filter(TABLE.outcome == "bc",
                sql.or_(TABLE.submitted == None, TABLE.submitted == 0),
                sql.or_(TABLE.dupe == None, TABLE.dupe == 0))\
        .order_by(TABLE.id)

    dupes = 0
    errs = set()
    for result in q:
        key = result.testbed_id, result.program_id

        if key in errs:
            dupes += 1
            result.dupe = 1
            continue
        errs.add(key)

        vendor = PLATFORMS_2_VENDORS[result.testbed.platform]
        outpath = fs.path(outbox, f"bug-report-{vendor}-{result.id}.c")            

        report = get_co_report(**{
            "result_id": result.id,
            "table": CLgenResult,
            "report_type": "bc",
        })

        with open(outpath, "w") as outfile:
            print(report, file=outfile)
        result.submitted = 1
        print(outpath)
print("done")
print(f"{dupes} duplicates flagged")

outbox/CLgenResults/bc/bug-report-intel-595928.c
outbox/CLgenResults/bc/bug-report-intel-595954.c
outbox/CLgenResults/bc/bug-report-intel-595997.c
outbox/CLgenResults/bc/bug-report-intel-596013.c
outbox/CLgenResults/bc/bug-report-nvidia-596047.c
outbox/CLgenResults/bc/bug-report-intel-596056.c
outbox/CLgenResults/bc/bug-report-intel-596092.c
outbox/CLgenResults/bc/bug-report-intel-596131.c
outbox/CLgenResults/bc/bug-report-intel-596212.c
outbox/CLgenResults/bc/bug-report-intel-596229.c
outbox/CLgenResults/bc/bug-report-intel-596440.c
outbox/CLgenResults/bc/bug-report-intel-596516.c
outbox/CLgenResults/bc/bug-report-intel-596550.c
outbox/CLgenResults/bc/bug-report-intel-596664.c
outbox/CLgenResults/bc/bug-report-intel-596705.c
outbox/CLgenResults/bc/bug-report-pocl-596759.c
outbox/CLgenResults/bc/bug-report-intel-596781.c
outbox/CLgenResults/bc/bug-report-intel-596813.c
outbox/CLgenResults/bc/bug-report-pocl-596911.c
outbox/CLgenResults/bc/bug-report-intel-596957.c
outbox/CLgenResults/b

outbox/CLgenResults/bc/bug-report-intel-606112.c
outbox/CLgenResults/bc/bug-report-intel-606369.c
outbox/CLgenResults/bc/bug-report-pocl-606415.c
outbox/CLgenResults/bc/bug-report-intel-606420.c
outbox/CLgenResults/bc/bug-report-nvidia-606535.c
outbox/CLgenResults/bc/bug-report-pocl-606580.c
outbox/CLgenResults/bc/bug-report-intel-606602.c
outbox/CLgenResults/bc/bug-report-intel-606707.c
outbox/CLgenResults/bc/bug-report-intel-606778.c
outbox/CLgenResults/bc/bug-report-intel-606908.c
outbox/CLgenResults/bc/bug-report-intel-606937.c
outbox/CLgenResults/bc/bug-report-intel-606998.c
outbox/CLgenResults/bc/bug-report-intel-607004.c
outbox/CLgenResults/bc/bug-report-intel-607281.c
outbox/CLgenResults/bc/bug-report-intel-607449.c
outbox/CLgenResults/bc/bug-report-intel-607522.c
outbox/CLgenResults/bc/bug-report-intel-607599.c
outbox/CLgenResults/bc/bug-report-intel-607607.c
outbox/CLgenResults/bc/bug-report-intel-607702.c
outbox/CLgenResults/bc/bug-report-intel-607743.c
outbox/CLgenResults/b

outbox/CLgenResults/bc/bug-report-intel-615538.c
outbox/CLgenResults/bc/bug-report-intel-615619.c
outbox/CLgenResults/bc/bug-report-intel-615628.c
outbox/CLgenResults/bc/bug-report-intel-615717.c
outbox/CLgenResults/bc/bug-report-intel-615729.c
outbox/CLgenResults/bc/bug-report-intel-615762.c
outbox/CLgenResults/bc/bug-report-intel-615824.c
outbox/CLgenResults/bc/bug-report-intel-615876.c
outbox/CLgenResults/bc/bug-report-intel-615891.c
outbox/CLgenResults/bc/bug-report-intel-615894.c
outbox/CLgenResults/bc/bug-report-intel-615898.c
outbox/CLgenResults/bc/bug-report-pocl-615902.c
outbox/CLgenResults/bc/bug-report-intel-615959.c
outbox/CLgenResults/bc/bug-report-intel-615961.c
outbox/CLgenResults/bc/bug-report-intel-616007.c
outbox/CLgenResults/bc/bug-report-intel-616012.c
outbox/CLgenResults/bc/bug-report-intel-616037.c
outbox/CLgenResults/bc/bug-report-intel-616072.c
outbox/CLgenResults/bc/bug-report-intel-616101.c
outbox/CLgenResults/bc/bug-report-intel-616142.c
outbox/CLgenResults/b

outbox/CLgenResults/bc/bug-report-intel-623760.c
outbox/CLgenResults/bc/bug-report-intel-623848.c
outbox/CLgenResults/bc/bug-report-intel-623932.c
outbox/CLgenResults/bc/bug-report-intel-623961.c
outbox/CLgenResults/bc/bug-report-pocl-623968.c
outbox/CLgenResults/bc/bug-report-pocl-624050.c
outbox/CLgenResults/bc/bug-report-intel-624054.c
outbox/CLgenResults/bc/bug-report-intel-624117.c
outbox/CLgenResults/bc/bug-report-intel-624125.c
outbox/CLgenResults/bc/bug-report-intel-624134.c
outbox/CLgenResults/bc/bug-report-intel-624357.c
outbox/CLgenResults/bc/bug-report-intel-624366.c
outbox/CLgenResults/bc/bug-report-pocl-624547.c
outbox/CLgenResults/bc/bug-report-intel-624657.c
outbox/CLgenResults/bc/bug-report-intel-624668.c
outbox/CLgenResults/bc/bug-report-intel-624676.c
outbox/CLgenResults/bc/bug-report-pocl-624786.c
outbox/CLgenResults/bc/bug-report-intel-624902.c
outbox/CLgenResults/bc/bug-report-intel-624947.c
outbox/CLgenResults/bc/bug-report-intel-624956.c
outbox/CLgenResults/bc/b

outbox/CLgenResults/bc/bug-report-pocl-643323.c
outbox/CLgenResults/bc/bug-report-intel-643407.c
outbox/CLgenResults/bc/bug-report-codeplay-643596.c
outbox/CLgenResults/bc/bug-report-codeplay-643705.c
outbox/CLgenResults/bc/bug-report-codeplay-643735.c
outbox/CLgenResults/bc/bug-report-codeplay-643758.c
outbox/CLgenResults/bc/bug-report-intel-643779.c
outbox/CLgenResults/bc/bug-report-intel-643916.c
outbox/CLgenResults/bc/bug-report-codeplay-643991.c
outbox/CLgenResults/bc/bug-report-intel-643994.c
outbox/CLgenResults/bc/bug-report-codeplay-644058.c
outbox/CLgenResults/bc/bug-report-intel-644112.c
outbox/CLgenResults/bc/bug-report-intel-644126.c
outbox/CLgenResults/bc/bug-report-codeplay-644141.c
outbox/CLgenResults/bc/bug-report-codeplay-644187.c
outbox/CLgenResults/bc/bug-report-pocl-644199.c
outbox/CLgenResults/bc/bug-report-intel-644240.c
outbox/CLgenResults/bc/bug-report-pocl-644322.c
outbox/CLgenResults/bc/bug-report-intel-644389.c
outbox/CLgenResults/bc/bug-report-codeplay-64458

outbox/CLgenResults/bc/bug-report-codeplay-658745.c
outbox/CLgenResults/bc/bug-report-codeplay-658871.c
outbox/CLgenResults/bc/bug-report-intel-658878.c
outbox/CLgenResults/bc/bug-report-codeplay-658957.c
outbox/CLgenResults/bc/bug-report-codeplay-658965.c
outbox/CLgenResults/bc/bug-report-pocl-658966.c
outbox/CLgenResults/bc/bug-report-codeplay-658985.c
outbox/CLgenResults/bc/bug-report-codeplay-659162.c
outbox/CLgenResults/bc/bug-report-intel-659253.c
outbox/CLgenResults/bc/bug-report-pocl-659283.c
outbox/CLgenResults/bc/bug-report-intel-659310.c
outbox/CLgenResults/bc/bug-report-pocl-659332.c
outbox/CLgenResults/bc/bug-report-codeplay-659459.c
outbox/CLgenResults/bc/bug-report-pocl-659558.c
outbox/CLgenResults/bc/bug-report-intel-659612.c
outbox/CLgenResults/bc/bug-report-intel-659620.c
outbox/CLgenResults/bc/bug-report-intel-659628.c
outbox/CLgenResults/bc/bug-report-codeplay-659646.c
outbox/CLgenResults/bc/bug-report-codeplay-659853.c
outbox/CLgenResults/bc/bug-report-pocl-659879.